In [1]:
import os
import tqdm
import ppscore
import numpy as np
import pandas as pd

In [2]:
path = "../data/causal/public/"
items = os.listdir(path)

In [3]:
df = pd.read_csv("../data/causal/public/ground_truth.csv", names=["label"])

In [4]:
df

,label
0,1
1,1
2,0
3,1
4,1
...,...
995,0
996,0
997,1
998,0


In [5]:
pred = []

for i in tqdm.trange(1000):
    p = pd.read_csv(f"../data/causal/public/{i}.csv", names=["x", "y"])
    res1 = ppscore.score(p, "x", "y")
    res2 = ppscore.score(p, "y", "x")

    if res1["model_score"] > res2["model_score"]:
        pred.append(1)
    else:
        pred.append(0)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:42<00:00, 23.57it/s]


In [8]:
df["pred"] = pred

In [9]:
df[df.label != df.pred]

,label,pred
3,1,0
6,1,0
9,1,0
24,0,1
25,1,0
...,...,...
971,1,0
975,1,0
988,0,1
991,0,1


In [10]:
len(df[df.label == df.pred]) / len(df)

0.886

In [11]:
p = pd.read_csv(f"../data/causal/public/3.csv", names=["x", "y"])
res1 = ppscore.score(p, "x", "y")
res2 = ppscore.score(p, "y", "x")

print(res1, res2)

{'x': 'x', 'y': 'y', 'ppscore': 0.4934563657295087, 'case': 'regression', 'is_valid_score': True, 'metric': 'mean absolute error', 'baseline_score': 0.15723851369418387, 'model_score': 0.07964816817394232, 'model': DecisionTreeRegressor()} {'x': 'y', 'y': 'x', 'ppscore': 0.5597553310292616, 'case': 'regression', 'is_valid_score': True, 'metric': 'mean absolute error', 'baseline_score': 0.1842330146228893, 'model_score': 0.08110760253613511, 'model': DecisionTreeRegressor()}


In [12]:
pred = []

for i in tqdm.trange(1000):
    p = pd.read_csv(f"../data/causal/private/{i}.csv", names=["x", "y"])
    res1 = ppscore.score(p, "x", "y")
    res2 = ppscore.score(p, "y", "x")

    if res1["model_score"] > res2["model_score"]:
        pred.append((i, 1))
    else:
        pred.append((i, 0))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:41<00:00, 23.89it/s]


In [13]:
submission = pd.DataFrame(data=pred, columns=["id", "direction"])

In [14]:
submission.to_csv("../predictions/t4_ppscore.csv", index=False)

In [15]:
scores = []

for i in tqdm.trange(1000):
    p = pd.read_csv(f"../data/causal/public/{i}.csv", names=["x", "y"])
    res1 = ppscore.score(p, "x", "y")
    res2 = ppscore.score(p, "y", "x")

    scores.append((
        res1["ppscore"],
        res1["baseline_score"],
        res1["model_score"],
        res2["ppscore"],
        res2["baseline_score"],
        res2["model_score"],
        res1["ppscore"] - res2["ppscore"],
        res1["baseline_score"] - res2["baseline_score"],
        res1["ppscore"] - res2["ppscore"]
    ))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:42<00:00, 23.63it/s]


In [16]:
from sklearn.linear_model import LogisticRegression

In [17]:
lr = LogisticRegression()
lr.fit(scores, df["label"])

LogisticRegression()

In [18]:
pred = lr.predict(scores)

In [19]:
np.sum(pred == df["label"]) / len(df)

0.811

In [24]:
img = pd.read_csv("../predictions/t4_img_clf.csv")

In [25]:
np.sum(img.direction == submission.direction)

477